In [42]:
import pandas as pd
import numpy as np

#from tqdm import tqdm,trange
from collections import defaultdict,Counter

import os
import re
import time
import io
import unicodedata as ud

import nltk

import warnings

In [82]:
with open('1205.txt','r',encoding='utf8') as file:
    text = ud.normalize('NFC',file.read()).split('\n')

script1 = [bn for bn in text if 'ca bệnh' in bn[:15].lower()]
[print(bn) for bn in script1]

- CA BỆNH BN3594 ghi nhận tại Thành phố Hà Nội: Bệnh nhân nữ, 8 tuổi, địa chỉ tại huyện Đông Anh, Thành phố Hà Nội; là F1 liên quan tới ổ dịch cũ, đã được cách ly.
- CA BỆNH BN3595 ghi nhận tại Thành phố Hà Nội: Bệnh nhân nữ, 59 tuổi, địa chỉ tại thị xã Kim Bảng, tỉnh Hà Nam; là F1 liên quan tới ổ dịch cũ, đã được cách ly.
- CA BỆNH BN3596 ghi nhận tại Thành phố Hà Nội: Bệnh nhân nam, 41 tuổi, địa chỉ tại huyện Thường Tín, Thành phố Hà Nội; là F1 liên quan tới ổ dịch cũ, đã được cách ly.
- CA BỆNH BN3597 ghi nhận tại Thành phố Hà Nội: Bệnh nhân nam, 38 tuổi, địa chỉ tại thành phố Thái Nguyên, tỉnh Thái Nguyên; là F1 liên quan tới ổ dịch cũ, đã được cách ly.
- CA BỆNH BN3598 ghi nhận tại Thành phố Hà Nội: Bệnh nhân nữ, 20 tuổi, địa chỉ tại huyện Phúc Thọ, Thành phố Hà Nội; là F1 liên quan tới ổ dịch cũ, đã được cách ly.
- CA BỆNH BN3609 ghi nhận tại Thành phố Hà Nội: Bệnh nhân nữ, 66 tuổi, địa chỉ tại huyện Yên Lạc, tỉnh Vĩnh Phúc; là F1 liên quan tới ổ dịch cũ, đã được cách ly.
- CA BỆ

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [47]:
patients = pd.DataFrame(columns=['id','sex','age','cbyt','thon','phuong','huyen','tp','quoctich','cluster','source','date','close contact','group'])

In [102]:
def get_id(bn):
    ids = re.findall(r'BN\w+-BN\w+|BN\w+',bn)
    ids = [i.replace('BN','') for i in ids]

    id_list = []
    for i in ids:
        numeric_ids = i.split('-')
        if len(numeric_ids)==2:
            id_list += list(np.arange(int(numeric_ids[0]),int(numeric_ids[1])))
        else:
            id_list.append(int(numeric_ids[0]))
    # Phòng việc nhầm mã với BN F0 (ví dụ: 'F1 của BN69'), thì chỉ nhận số bệnh nhân theo thứ tự tăng dần, 
    # vì chắc chắc không ai liệt kê kiểu đi lùi hay 'bn50 là f1 của bn49' cả
    return [id_list[0]]+[id_list[i] for i in range(1,len(id_list)) if id_list[i-1]<id_list[i]]

In [103]:
# nếu từ nam với nữ trùng địa danh (Hà Nam, Nữ Oa,...), việc chỉ lọc chữ thường sẽ giúp bỏ được các trường hợp trên 
get_sex = lambda bn: 'Nữ' if 'nữ' in bn else 'Nam' if 'nam' in bn else np.nan
get_age = lambda bn: int(re.search(r'[0-9]+',re.findall(r'[0-9]+ tuổi',bn)[0]).group())
get_cbyt = lambda bn: int('cán bộ y tế' in bn or 'nhân viên y tế' in bn)

In [176]:
def get_location(don_vi_hanh_chinh, bn):
    regex = re.compile('|'.join([don_vi_hanh_chinh+' (.*?)'+stop for stop in [',','\\.',';',':']]))
    # Nếu ghi nhận 2 địa điểm thì chọn địa điểm đầu, vì sẽ nêu địa chỉ trước lịch sử di chuyển
    loc_text = re.findall(regex, bn)[0][0]
    loc = re.split(r';|,|\.|:',loc_text)[0]
    return loc

In [179]:
def get_quoctich(bn):
    regex = re.compile('|'.join(['quốc tịch'+' (.*?)'+stop for stop in [',','\\.',';',':']]))
    # Nếu ghi nhận 2 địa điểm thì chọn địa điểm đầu, vì sẽ nêu địa chỉ trước lịch sử di chuyển
    loc_text = re.findall(regex, bn.lower())[0][0]
    loc = re.split(r';|,|\.|:',loc_text)
    return 'Vietnam' if not loc else loc[0]
#get_quoctich()

In [177]:
get_location('tỉnh',script1[12])

'Bà Rịa - Vũng Tàu'

In [151]:
'a,'.split(',')

['a', '']

In [44]:
def add_patient_info(bn):
    patients.loc[i] = [...]
    pass

b'- ca b\xe1\xbb\x87nh bn3'


b'ca b\xe1\xbb\x87nh'